## Import the necessary libraries

In [13]:
import pandas as pd
import numpy as np
import nltk
import string
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

from sklearn.metrics import f1_score

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

## Load in your data from kaggle.  
By working in a kaggle kernel, you can access the data directly from the competition, as well as make your submission without downloading your output file

In [14]:
train = pd.read_csv('../input/climate-change-edsa2020-21/train.csv')
test = pd.read_csv('../input/climate-change-edsa2020-21/test.csv')

## Data Pre-processing

#mystring = 'hello'
#empty= '   '
#mystring.isspace()
#empty.isspace()
#blanks = []  # start with an empty list

#for i,st,ms,tw in train.itertuples():  # iterate over the DataFrame
#    if type(st)==str:            # avoid NaN values
        #if st.isspace():         # test 'review' for whitespace
            #blanks.append(i)     # add matching index numbers to the list
        
#print(len(blanks), 'blanks: ', blanks)

In [16]:
len(train)

15819

In [17]:
train.sentiment.value_counts()

 1    8530
 2    3640
 0    2353
-1    1296
Name: sentiment, dtype: int64

## Splitting out the X variable from the target

In [18]:
y = train['sentiment']
#X = train['CleanMessage']
X= train['message']

## Turning text into something your model can read

In [19]:
vectorizer = TfidfVectorizer(ngram_range=(1,2), min_df=2, stop_words="english")
vectorizer = TfidfVectorizer()
X_vectorized = vectorizer.fit_transform(X)

## Splitting the training data into a training and validation set

In [20]:
from sklearn.model_selection import RepeatedKFold
kf = RepeatedKFold(n_splits=5, n_repeats=10, random_state=None) 

for train_index, test_index in kf.split(X_vectorized):
    
    X_train, X_test = X_vectorized[train_index], X_vectorized[test_index] 
    y_train, y_test = y[train_index], y[test_index]

In [21]:
from imblearn.over_sampling import SMOTE
smote =SMOTE()
X_smote,y_smote =smote.fit_sample(X_train,y_train)

In [22]:
from collections import Counter
print ("Before smote",Counter(y_train))
print ("after smote",Counter(y_smote))

Before smote Counter({1: 6836, 2: 2888, 0: 1897, -1: 1035})
after smote Counter({1: 6836, 2: 6836, 0: 6836, -1: 6836})


## Training the model and evaluating using the validation set 

In [23]:
#rfc = RandomForestClassifier(n_estimators=200, random_state=44)
rfc = RandomForestClassifier()
rfc.fit(X_smote, y_smote)
rfc_pred = rfc.predict(X_test)

## Checking the performance of our model on the validation set

In [24]:
f1_score(y_test, rfc_pred,average="macro")

0.6199574560962525

In [25]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
#print(confusion_matrix(y_val, rfc_pred))
print(classification_report(y_test, rfc_pred))
#print(accuracy_score(y_val, rfc_pred))

              precision    recall  f1-score   support

          -1       0.88      0.26      0.40       261
           0       0.60      0.45      0.51       456
           1       0.74      0.87      0.80      1694
           2       0.78      0.77      0.77       752

    accuracy                           0.73      3163
   macro avg       0.75      0.59      0.62      3163
weighted avg       0.74      0.73      0.72      3163



## Getting our test set ready 

In [26]:
testx = test['message']
test_vect = vectorizer.transform(testx)

## Making predictions on the test set and adding a sentiment column to our original test df

In [27]:
y_pred = rfc.predict(test_vect)

In [28]:
test['sentiment'] = y_pred

In [29]:
test.head()

,message,tweetid,sentiment
0,Europe will now be looking to China to make su...,169760,1
1,Combine this with the polling of staffers re c...,35326,1
2,"The scary, unimpeachable evidence that climate...",224985,1
3,@Karoli @morgfair @OsborneInk @dailykos \nPuti...,476263,1
4,RT @FakeWillMoore: 'Female orgasms cause globa...,872928,0


## Creating an output csv for submission

In [30]:
test[['tweetid','sentiment']].to_csv('testsubmission.csv', index=False)